In [1]:
# 1- Importing necessary libraries and defining functions.
# Opening the retina image .tif file
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import tifffile
from tifffile import imread
from skimage.measure import label   
from skimage.morphology import skeletonize, skeletonize_3d
from skimage import morphology, filters
from skan import Skeleton, summarize
from skan import draw
from skimage.draw import polygon
import napari
import os

def modified_max_inscribed_circle(bw, f):    
    D = ndimage.distance_transform_edt(bw)
    Rs = -np.sort(-D, axis=None)
    R = Rs[0]
    RInds = np.argsort(-D, axis=None)
    RInds = RInds[Rs >= f*R]
    [cy, cx] = np.unravel_index(RInds, D.shape)
    return R, cx, cy

def create_circular_mask(h, w, center=None, radius=None):
    if center is None: 
        center = (int(w/2), int(h/2))
    if radius is None: 
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

In [2]:
# 2- Opening .tif retina image file in "retina_sample" folder
file_path = "retina_sample/"
folder_names = os.listdir(file_path)

# To open the files one by one in this folder you can siiply change the "0" value.
# im1= imread(file_path+folder_names[1]) for example opens the second image file
# im1= imread(file_path+folder_names[2]) opens the third image file
# Instead yoo can write the file name directly: im1= imread("wt_retina.tif")
im1= imread(file_path+folder_names[0])

image_array1 = np.array(im1)
print(image_array1.shape)
print(folder_names)
viewer = napari.Viewer()
viewer.add_image(image_array1)

In [1]:
# Now, open Napari viewer screen and draw a polygon, which includes all the retina area you want to analyze!
# Then, after that continue running the next section.

In [3]:
# 3- Creating a filled polygon to draw a central bead:
shapes = viewer.layers["Shapes"].data
substitute_sprout = np.zeros((image_array1.shape[0], image_array1.shape[1]), 'uint8')
for i in shapes:
    polygon1 = []
    for ii in i:    
        polygon1.append(ii)
    for iii in polygon1:
        int_array = i.astype(int)
        listem = []
    for iiii in int_array:
        a = iiii[0]
        b = iiii[1]
        listem.append((a,b))
        listem_array = np.array(listem)        

    poly = listem_array
    rr, cc = polygon(poly[:,0], poly[:,1])
substitute_sprout[rr,cc] = 255
sprout_subs = np.array(substitute_sprout)
plt.imshow(sprout_subs)

In [4]:
# 4- Circular shape removal
R, cx, cy = modified_max_inscribed_circle(sprout_subs, f=0.9)
im_med = scipy.ndimage.median_filter(image_array1, size=(6,6)) 
sz = im_med.shape
bead_03 = np.zeros(sz, dtype=bool)
bead_05 = np.zeros(sz, dtype=bool)
bead_07 = np.zeros(sz, dtype=bool)
bead_1 = np.zeros(sz, dtype=bool)

for i in range(0, len(cx), len(cx)//2):
    # You can change radius to R*n in order to get the correct seperation of sprouts
    circ_03 = create_circular_mask(sz[0], sz[1], center=(cx[i],cy[i]), radius=R*0.3)
    bead_03 = np.logical_or(bead_03, circ_03)
    circ_05 = create_circular_mask(sz[0], sz[1], center=(cx[i],cy[i]), radius=R*0.5)
    bead_05 = np.logical_or(bead_05, circ_05)
    circ_07 = create_circular_mask(sz[0], sz[1], center=(cx[i],cy[i]), radius=R*0.7)
    bead_07 = np.logical_or(bead_07, circ_07)
    
    circ_1 = create_circular_mask(sz[0], sz[1], center=(cx[i],cy[i]), radius=R*1)
    bead_1 = np.logical_or(bead_1, circ_1)

bead_removed_03 = im_med.copy()
bead_removed_05 = im_med.copy()
bead_removed_07 = im_med.copy()

bead_removed_03[bead_03==1]=0
bead_removed_05[bead_05==1]=0
bead_removed_07[bead_07==1]=0
"""
branches_03 = bead_removed_03 > filters.threshold_otsu(bead_removed_03)
branches_05 = bead_removed_05 > filters.threshold_otsu(bead_removed_05)
branches_07 = bead_removed_07 > filters.threshold_otsu(bead_removed_07)
"""

# if the thresholding does not look good, put manual threshold values
# print(filters.threshold_otsu(bead_removed_05))

branches_03 = bead_removed_03 > 50
branches_05 = bead_removed_05 > 50
branches_07 = bead_removed_07 > 50

viewer.add_image(branches_03)
viewer.add_image(branches_05)
viewer.add_image(branches_07)

In [10]:
# 5- BRANCH LENGTH ANALYSIS
skeleton_scikit03 = skeletonize(branches_03)
skeleton_scikit05 = skeletonize(branches_05)
skeleton_scikit07 = skeletonize(branches_07)

branch_data03 = summarize(Skeleton(skeleton_scikit03))
print("skeleton_length_03:", branch_data03["branch-distance"].sum())
print("total_branch_number_03:", len(branch_data03["branch-distance"]))
print("Long_branch_number_03:", len(branch_data03[branch_data03["branch-distance"]>40]))

branch_data05 = summarize(Skeleton(skeleton_scikit05))
print("skeleton_length_05:", branch_data05["branch-distance"].sum())
print("total_branch_number_05:", len(branch_data05["branch-distance"]))
print("Long_branch_number_05:", len(branch_data05[branch_data05["branch-distance"]>40]))

branch_data07 = summarize(Skeleton(skeleton_scikit07))
print("skeleton_length_07:", branch_data07["branch-distance"].sum())
print("total_branch_number_07:", len(branch_data07["branch-distance"]))
print("Long_branch_number_07:", len(branch_data07[branch_data07["branch-distance"]>40]))

# showing the histograms of branch analysis
branch_data03.hist(column='branch-distance', bins=100, range=[0,250])
branch_data05.hist(column='branch-distance', bins=100, range=[0,250])
branch_data07.hist(column='branch-distance', bins=100, range=[0,250])

viewer.add_image(skeleton_scikit03)
viewer.add_image(skeleton_scikit05)
viewer.add_image(skeleton_scikit07)

In [12]:
# VASCULAR DENSITY:
# Vascular density = vessel area / total area:
filled_03 = ndimage.binary_fill_holes(branches_03) * 1
filled_05 = ndimage.binary_fill_holes(branches_05) * 1
filled_07 = ndimage.binary_fill_holes(branches_07) * 1

counting_branches_03 = np.count_nonzero(branches_03)
counting_filled_03 = np.count_nonzero(filled_03)
vessel_density_03 = counting_branches_03 / counting_filled_03*100
print("vessel_density_03:", vessel_density_03)

counting_branches_05 = np.count_nonzero(branches_05)
counting_filled_05 = np.count_nonzero(filled_05)
vessel_density_05 = counting_branches_05 / counting_filled_05*100
print("vessel_density_05:", vessel_density_05)

counting_branches_07 = np.count_nonzero(branches_07)
counting_filled_07 = np.count_nonzero(filled_07)
vessel_density_07 = counting_branches_07 / counting_filled_07*100
print("vessel_density_07:", vessel_density_07)

# radial expansion:
# the mean distance covered by the vessels growing from the optic nerve
print("radial expansion", R)